In [1]:
import torch
from tqdm import tqdm
from pytorch_pretrained_bert import GPT2Tokenizer, GPT2Model, GPT2LMHeadModel

# OPTIONAL: if you want to have more information on what's happening, activate the logger as follows
import logging
logging.basicConfig(level=logging.INFO)

device = "cuda:0" if torch.cuda.is_available() else "cpu"

Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex.


In [2]:
# Load pre-trained model tokenizer (vocabulary)
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

# Encode some inputs
text_1 = "Once upon a time, in a computer lab not particularly far away, there was a guy. An awesome, cool, badass guy, yes; but just a guy nonetheless. Surprisingly, this story thingy is not about the guy, and the fact that he oozes awesome; Although that is far more interesting. It is in fact, about a girl who is yet to enter the lab. Apparently, the guy had something she wanted. Few sheets of papers, that were hers' to begin with. And so she walks up to him and calls him. He was writing something and minding his own business. He’s plenty annoyed at hearing his name. It was only moments ago that another girl had interrupted him.  Now, he happens to be a pretty weird guy, always thinking of something else entirely when while doing mundane tasks like writing or driving or walking. And so as usual he was apne thoughts mein dooba hua. Until he saw her. You see, seeing her had a weird effect on him. As if his mind lost its train of thought immediately. A new train had arrived, and it went ‘oh. She pretty’."
text_2 = "Now, he had seen her before. Multiple times, she looked plenty normal to him. Even earlier that morning, when he asked for the math tutorials she wanted back. Surely she didn't look like this subhe ko. He would have noticed. Her eyes were dark and brown and she had these cute little eyebrows knitted together in a frown; a perfectly placed birthmark above her mouth. It was this stupid birthmark that demanded the most attention. She seemed distracted, stuttering a bit and her eyes kept darting away. For some reason, she leaned towards him as she listened, with her lips slightly parted, making her even more distracting. Probably because he wasn't audible enough. She just stood there, apparently oblivious to all the distraction she was causing. Probably because the guy looked all calm and composed, regardless of what was going on in his head. Hopefully. I mean, of course he was. He's the type that walks away calmly from explosions. Like the lead of a spy movie. And when he finally thought he succeeded in ignoring her face. He noticed this cute not particularly small birthmark, the same color as the stripes on her tee. He could see it, for she was standing and he was sitting."
indexed_tokens_1 = tokenizer.encode(text_1)
indexed_tokens_2 = tokenizer.encode(text_2)

# Convert inputs to PyTorch tensors
tokens_tensor_1 = torch.tensor([indexed_tokens_1])
tokens_tensor_2 = torch.tensor([indexed_tokens_2])

INFO:pytorch_pretrained_bert.tokenization_gpt2:loading vocabulary file https://s3.amazonaws.com/models.huggingface.co/bert/gpt2-vocab.json from cache at C:\Users\dhruv\.pytorch_pretrained_bert\f2808208f9bec2320371a9f5f891c184ae0b674ef866b79c58177067d15732dd.1512018be4ba4e8726e41b9145129dc30651ea4fec86aa61f4b9f40bf94eac71
INFO:pytorch_pretrained_bert.tokenization_gpt2:loading merges file https://s3.amazonaws.com/models.huggingface.co/bert/gpt2-merges.txt from cache at C:\Users\dhruv\.pytorch_pretrained_bert\d629f792e430b3c76a1291bb2766b0a047e36fae0588f9dbc1ae51decdff691b.70bec105b4158ed9a1747fea67a43f5dee97855c64d62b6ec3742f4cfdb5feda


In [3]:
# Load pre-trained model (weights)
model = GPT2LMHeadModel.from_pretrained('gpt2')
model.eval()

# If you have a GPU, put everything on cuda
tokens_tensor_1 = tokens_tensor_1.to(device)
tokens_tensor_2 = tokens_tensor_2.to(device)
model.to(device)

# Predict all tokens
with torch.no_grad():
    predictions_1, past = model(tokens_tensor_1)
    # past can be used to reuse precomputed hidden state in a subsequent predictions
    # (see beam-search examples in the run_gpt2.py example).
    predictions_2, past = model(tokens_tensor_2, past=past)

# get the predicted last token
# predicted_index = torch.argmax(predictions_2[0, -1, :]).item()
# predicted_token = tokenizer.decode([predicted_index])

pred = torch.argmax(predictions_1, dim=2)[0].cpu().numpy().tolist()
pred_text = tokenizer.decode(pred)

INFO:pytorch_pretrained_bert.modeling_gpt2:loading weights file https://s3.amazonaws.com/models.huggingface.co/bert/gpt2-pytorch_model.bin from cache at C:\Users\dhruv\.pytorch_pretrained_bert\4295d67f022061768f4adc386234dbdb781c814c39662dd1662221c309962c55.778cf36f5c4e5d94c8cd9cefcf2a580c8643570eb327f0d4a1f007fab2acbdf1
INFO:pytorch_pretrained_bert.modeling_gpt2:loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/gpt2-config.json from cache at C:\Users\dhruv\.pytorch_pretrained_bert\4be02c5697d91738003fb1685c9872f284166aa32e061576bbe6aaeb95649fcf.085d5f6a8e7812ea05ff0e6ed0645ab2e75d80387ad55c1ad9806ee70d272f80
INFO:pytorch_pretrained_bert.modeling_gpt2:Model config {
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "n_ctx": 1024,
  "n_embd": 768,
  "n_head": 12,
  "n_layer": 12,
  "n_positions": 1024,
  "vocab_size": 50257
}



In [4]:
predictions_1.shape

torch.Size([1, 238, 50257])

In [5]:
torch.cuda.is_available()

True

In [6]:
pred_text

' the a time, the the time game, far well from, a was a man who He old guy brilliant guy cool guy. who, but he as guy..\n, he guy is was was not a him guy. but it guy that it\'ses a. it,\'s a from important than\n\'s about fact about a the guy who is a to be the world, She, she girl is been to wanted to She things of paper, but is all,, keep with. She she, was into to him, starts out a "\'s a a, sheing his own business.\n was�s writing of, her her name called He� a a before that he girl had called him. He\xa0, she� to be a girl good guy. and a about himself... he he his things. this. reading. whatever. He he, he, is justathetico ofandering.e.an.  he was her. He know, he her, been lot effect on him. He he he mind was its ability of thought.. He girl thought of arrived. and he was off� �,  was much�s He'

In [7]:
predictions_2

tensor([[[-128.1211, -131.0129, -134.6268,  ..., -140.1218, -134.4016,
          -133.3209],
         [-163.7341, -165.2466, -166.9875,  ..., -170.0072, -166.5462,
          -164.6214],
         [-144.6157, -144.9538, -148.4446,  ..., -155.3010, -143.6271,
          -147.5490],
         ...,
         [-150.4981, -151.3717, -155.9007,  ..., -160.9858, -151.3975,
          -153.6461],
         [-126.8530, -129.2441, -135.1330,  ..., -140.6560, -131.7047,
          -131.3890],
         [-157.5303, -155.5184, -157.6709,  ..., -168.9378, -165.0819,
          -152.4198]]], device='cuda:0')

In [10]:
indexed_tokens_1

[7454,
 2402,
 257,
 640,
 11,
 287,
 257,
 3644,
 2248,
 407,
 3573,
 1290,
 1497,
 11,
 612,
 373,
 257,
 3516,
 13,
 1052,
 7427,
 11,
 3608,
 11,
 37289,
 3516,
 11,
 3763,
 26,
 475,
 655,
 257,
 3516,
 19018,
 13,
 47183,
 11,
 428,
 1621,
 1517,
 88,
 318,
 407,
 546,
 262,
 3516,
 11,
 290,
 262,
 1109,
 326,
 339,
 49564,
 274,
 7427,
 26,
 4900,
 326,
 318,
 1290,
 517,
 3499,
 13,
 632,
 318,
 287,
 1109,
 11,
 546,
 257,
 2576,
 508,
 318,
 1865,
 284,
 3802,
 262,
 2248,
 13,
 18626,
 11,
 262,
 3516,
 550,
 1223,
 673,
 2227,
 13,
 20463,
 15747,
 286,
 9473,
 11,
 326,
 547,
 25144,
 6,
 284,
 2221,
 351,
 13,
 843,
 523,
 673,
 11114,
 510,
 284,
 683,
 290,
 3848,
 683,
 13,
 679,
 373,
 3597,
 1223,
 290,
 2000,
 278,
 465,
 898,
 1597,
 13,
 679,
 447,
 247,
 82,
 6088,
 25602,
 379,
 4854,
 465,
 1438,
 13,
 632,
 373,
 691,
 7188,
 2084,
 326,
 1194,
 2576,
 550,
 19072,
 683,
 13,
 220,
 2735,
 11,
 339,
 4325,
 284,
 307,
 257,
 2495,
 7650,
 3516,
 11,
 1464,
 3